In [150]:
import pandas as pd
import numpy as np

In [151]:
df2 = pd.read_csv('../scraped-data-2024/cluster2-html.csv')
df2 = pd.concat([df2, pd.read_csv('../scraped-data-2024/cluster2_5-pdf.csv')])

In [152]:
df2.replace('\xa0', np.nan, inplace=True)
df2.dropna(subset=["Pontuação Regata"], inplace=True)
df2['Punição'] = 0
df2['Descarte'] = 0

In [153]:
import re
import unidecode
import numpy as np

def process_string(s):
    # Remove espaços em branco extras
    s = str(s).strip()
    
    numbers = re.findall(r'\d+(?:\.\d+)?', s)
    if numbers == []:
        numbers.append(np.nan)
    
    has_parentheses = 1 if '(' in s or ')' in s else 0
    
    letters = ''.join(re.findall(r'[a-zA-Z]', s))
    if letters == '':
        letters = 0
    return numbers, has_parentheses, letters

In [154]:
df2['Punição'] = df2['Pontuação Regata'].apply(lambda x: process_string(x)[2])
df2['Descarte'] = df2['Pontuação Regata'].apply(lambda x: process_string(x)[1])
df2['Pontuação Regata'] = df2['Pontuação Regata'].apply(lambda x: process_string(x)[0][0])
df2["Nome Competidor"] = df2["Nome Competidor"] .apply(lambda x: unidecode.unidecode(x).upper())

In [155]:
df2.to_csv('../cleaned_data_2024/cluster_2.csv', index=False)

In [156]:
df3 = pd.read_csv('../scraped-data-2024/cluster3.csv')

In [157]:
df3["Nome Competidor"] = df3["Nome Competidor"].apply(lambda x: str(x).strip().replace('/', ', '))

In [158]:
# df3.to_csv('../cleaned_data_2024/cluster_3.csv', index=False)

In [159]:
df5 = pd.read_csv("../scraped-data-2024/cluster5-pdf.csv")

In [160]:
def processar_nomes(nome):
    # Substitui um espaço entre uma letra minúscula e uma letra maiúscula por vírgula
    return re.sub(r'([a-z])([A-Z])', r'\1, \2', nome)

In [163]:
df5['Pontuação Regata'] = df5['Pontuação Regata'].apply(lambda x: processar_nomes(x))
df5

,Nome Competidor,ID Competição,Classe Vela,Pontuação Regata,Flotilha,Posição Geral,Punição,Pontuação Total,Nett,Nome Competição,Regata
0,DAHLBERG Anton KARLSSON Lovisa,7,470,1,G,1,NaN,45,35,Semaine Olympique Francaise De Voile 2022,R1
1,MACHETTI Hippolyte RETORNAZ Aloise,7,470,2,G,2,NaN,54,42,Semaine Olympique Francaise De Voile 2022,R1
2,WANSER Luise AUTENRIETH Philipp,7,470,5,G,3,NaN,66,51,Semaine Olympique Francaise De Voile 2022,R1
3,FERRARI Giacomo CARUSO Bianca,7,470,3,G,4,NaN,68,53,Semaine Olympique Francaise De Voile 2022,R1
4,VADLAU Lara MAHR Lukas,7,470,7,G,5,NaN,68,54,Semaine Olympique Francaise De Voile 2022,R1
...,...,...,...,...,...,...,...,...,...,...,...
2828,UGOLINI Gianluigi GIUBILEI Maria,7,Nacra,3.0,G,6,NaN,149,116,Semaine Olympique Francaise De Voile 2022,R16 MR
2829,MAJDALANI Mateo BOSCO Eugenia,7,Nacra,1.0,G,7,NaN,149,133,Semaine Olympique Francaise De Voile 2022,R16 MR
2830,SAOUMA-PEDERSEN Natac BORRESKOV Mathias Bruun,7,Nacra,2.0,G,8,NaN,152,136,Semaine Olympique Francaise De Voile 2022,R16 MR
2831,VAN DER MEER Laila BOUWER Bjarne,7,Nacra,9.0,G,9,NaN,158,136,Semaine Olympique Francaise De Voile 2022,R16 MR


In [162]:
# df5['Punição'] = df5['Pontuação Regata'].apply(lambda x: process_string(x)[2])
# df5['Descarte'] = df5['Pontuação Regata'].apply(lambda x: process_string(x)[1])
# df5['Pontuação Regata'] = df5['Pontuação Regata'].apply(lambda x: process_string(x)[0][0])
# df5["Nome Competidor"] = df2["Nome Competidor"] .apply(lambda x: unidecode.unidecode(x).upper())
# df5['Pontuação Regata'].apply(lambda x: process_string(x)[0])[100] == []
df5.tail()

,Nome Competidor,ID Competição,Classe Vela,Pontuação Regata,Flotilha,Posição Geral,Punição,Pontuação Total,Nett,Nome Competição,Regata
2828,UGOLINI Gianluigi GIUBILEI Maria,7,Nacra,3.0,G,6,NaN,149,116,Semaine Olympique Francaise De Voile 2022,R16 MR
2829,MAJDALANI Mateo BOSCO Eugenia,7,Nacra,1.0,G,7,NaN,149,133,Semaine Olympique Francaise De Voile 2022,R16 MR
2830,SAOUMA-PEDERSEN Natac BORRESKOV Mathias Bruun,7,Nacra,2.0,G,8,NaN,152,136,Semaine Olympique Francaise De Voile 2022,R16 MR
2831,VAN DER MEER Laila BOUWER Bjarne,7,Nacra,9.0,G,9,NaN,158,136,Semaine Olympique Francaise De Voile 2022,R16 MR
2832,SVENSSON Ida DACKHAMMAR Marcus,7,Nacra,8.0,G,10,NaN,179,156,Semaine Olympique Francaise De Voile 2022,R16 MR
